# Imports

In [2]:
import numpy as np
import cv2
import torch
import torchvision
import cvzone
from facenet_pytorch import MTCNN, InceptionResnetV1
device = 'cuda' if torch.cuda.is_available() else 'cpu'
classes_names =[]
with open('./classes-vgg.txt', 'r') as f:
    classes_names = f.read().splitlines() 
len(classes_names)

91

In [3]:
image_path = './Images/Human/2048213_europeana_fashion_h.webp'
img_p = './Images/000035.jpg'

In [88]:
down_width = 800
down_height = 1000
down_points = (down_width, down_height)

In [3]:
model = torchvision.models.detection.ssd300_vgg16(weights=torchvision.models.detection.SSD300_VGG16_Weights.COCO_V1)
model.eval()

SSD(
  (backbone): SSDFeatureExtractorVGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=

In [16]:
img = cv2.imread(img_p)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#img = cv2.resize(img, down_points, interpolation= cv2.INTER_LINEAR)
image = img.copy()
img_t = torchvision.transforms.ToTensor()
with torch.no_grad():
    results = model([img_t(img)])
    bbox, scores, labels = results[0]['boxes'], results[0]['scores'], results[0]['labels']
    nums = torch.argwhere(scores > 0.50).shape[0]
    print(nums)
    for i in range(nums):
        x, y, w, h = bbox[i].numpy().astype('int')
        cv2.rectangle(image, (x, y), (w, h), (0,0,255), 2)
        classIDX = labels[i].numpy().astype('int')
        cvzone.putTextRect(image, classes_names[classIDX-1], [x, y+10], scale=1, thickness =1)
        print(labels[i])
cv2.imshow('frame', image)

cv2.waitKey(0)

8
tensor(1)
tensor(47)
tensor(47)
tensor(1)
tensor(47)
tensor(47)
tensor(47)
tensor(47)


-1

# With Camera


In [10]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print( "Error: Could not open camera" )

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame")
        break
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = img.copy()
    img_t = torchvision.transforms.ToTensor()

    with torch.no_grad():
        results = model([img_t(img)])
        bbox, scores, labels = results[0]['boxes'], results[0]['scores'], results[0]['labels']
        nums = torch.argwhere(scores > 0.80).shape[0]
        #print(nums)
        for i in range(nums):                                   # Iterate over identified objects
            x, y, w, h = bbox[i].numpy().astype('int')          # Bbox for plot
            cv2.rectangle(frame, (x, y), (w, h), (0,0,255), 5)  # Draw Bbox
            classIDX = labels[i].numpy().astype('int')          # Index of class in the classes list        
            idf_class = classes_names[classIDX-1]               # Identified Class
            # Format string Draw Cv2
            f_score = format(float(scores[i]), '.2f')
            text = idf_class + " Score:" +str(f_score) 
            # Draw bboxand text

            # If person is detected
            if(idf_class == 'person'):
                text += " DANGER"
            
            cvzone.putTextRect(frame, text, [x, y+10], scale=1, thickness =1)
    #print(labels[i])
    
    cv2.imshow('Webcam', frame)

    if( cv2.waitKey(1) & 0xFF == ord('q')):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
def process_idf_class( cl ):
    if (cl == 'person'):
        pass
    elif cl == 'dog':
        pass
    elif cl == ''